<a href="https://colab.research.google.com/github/szakmarynandor/FAKT-MLCourse/blob/main/hazi04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. házi feladat
Apple árfolyam előrejelzés (napi záró árfolyamok)

In [150]:
import numpy as np
import pandas as pd
from tensorflow import keras

from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Flatten, Dense, Dropout, Conv1D, MaxPooling1D, AveragePooling1D
import keras

In [151]:
from google.colab import drive
from google.colab import files

In [152]:
drive.mount('/content/drive')
apple = pd.read_csv('/content/drive/My Drive/FAKT_ML/apple.csv', usecols=[1])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
apple.head()

,AAPL.Close
0,26.337500
1,25.677500
2,25.174999
3,24.112499
4,24.240000


In [154]:
apple.shape

(1559, 1)

In [155]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	x, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps # Az adott szekvencia-darab utolsó eleme
		if end_ix > len(sequence)-1: # Ellenőrizzük nem értünk-e a teljes lista végére
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # Kimentjük a szekvencia darabot és az utolsó elemet mint eredményváltozó
		x.append(seq_x)
		y.append(seq_y)
	return np.array(x), np.array(y)

In [156]:
x, y = split_sequence(apple['AAPL.Close'], 10)

In [157]:
x.shape

(1549, 10)

In [158]:
x

array([[ 26.3375  ,  25.6775  ,  25.174999, ...,  24.3475  ,  24.879999,
         24.282499],
       [ 25.6775  ,  25.174999,  24.112499, ...,  24.879999,  24.282499,
         24.165001],
       [ 25.174999,  24.112499,  24.24    , ...,  24.282499,  24.165001,
         24.1975  ],
       ...,
       [160.070007, 162.740005, 164.850006, ..., 163.169998, 159.300003,
        157.440002],
       [162.740005, 164.850006, 165.119995, ..., 159.300003, 157.440002,
        162.949997],
       [164.850006, 165.119995, 163.199997, ..., 157.440002, 162.949997,
        158.520004]])

In [159]:
y.shape

(1549,)

In [160]:
y

array([ 24.165001,  24.1975  ,  24.075001, ..., 162.949997, 158.520004,
       154.729996])

In [161]:
#kiíratom
#for i in range(len(x)):
#	print(x[i], y[i])
#inkább mégsem, mert nagyon hosszú :)

In [162]:
from sklearn.model_selection import train_test_split

In [163]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

###Normalizálás

In [164]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [165]:
normalizer = StandardScaler()

In [166]:
normalizer.fit(x_train)
x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)

In [167]:
x_test

array([[-0.9419193 , -0.94349536, -0.93979129, ..., -0.92515193,
        -0.9223526 , -0.92322473],
       [ 0.04867653,  0.07204759, -0.03785431, ...,  0.06331157,
         0.03859622, -0.09133467],
       [-0.69692954, -0.70809979, -0.64959842, ..., -0.60345466,
        -0.62347169, -0.62734459],
       ...,
       [ 1.7263752 ,  1.70590693,  1.73449196, ...,  1.69056257,
         1.6891968 ,  1.67694938],
       [-0.37453892, -0.37686085, -0.37319198, ..., -0.34692841,
        -0.34449383, -0.33426317],
       [-0.07760906, -0.07242406, -0.07870494, ..., -0.09753709,
        -0.08020488, -0.08528951]])

In [174]:
input = Input(shape=(10,1))

x=Conv1D(filters=256, kernel_size=3, strides=1, activation='relu')(input)
x=MaxPooling1D(pool_size=2, strides=1)(x)
x=Conv1D(filters=256, kernel_size=3, strides=1, activation='relu')(x)
x=MaxPooling1D(pool_size=2, strides=1)(x)

x=Flatten()(x)

x=Dense(4096, activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(4096, activation='relu')(x)
x=Dropout(0.3)(x)

output = Dense(1, activation='linear')(x)

In [176]:
AlexNet=keras.Model(input,output)

In [184]:
AlexNet.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])

In [185]:
AlexNet.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 conv1d_17 (Conv1D)          (None, 8, 256)            1024      
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 7, 256)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 5, 256)            196864    
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 4, 256)           0         
 g1D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 1024)              0  

In [186]:
from tensorflow.keras.callbacks import EarlyStopping

In [187]:
early_stopping_callback = EarlyStopping(patience=20, restore_best_weights=True)

In [188]:
AlexNet.fit(x_train, y_train, epochs=200, validation_split=0.10, callbacks=[early_stopping_callback])

Epoch 1/200
35/35 [==============================] - 3s 47ms/step - loss: 182.6049 - accuracy: 0.0000e+00 - val_loss: 16.3803 - val_accuracy: 0.0000e+00
Epoch 2/200
35/35 [==============================] - 1s 22ms/step - loss: 15.2181 - accuracy: 0.0000e+00 - val_loss: 7.1167 - val_accuracy: 0.0000e+00
Epoch 3/200
35/35 [==============================] - 1s 21ms/step - loss: 13.2803 - accuracy: 0.0000e+00 - val_loss: 7.1136 - val_accuracy: 0.0000e+00
Epoch 4/200
35/35 [==============================] - 1s 21ms/step - loss: 13.1552 - accuracy: 0.0000e+00 - val_loss: 6.7107 - val_accuracy: 0.0000e+00
Epoch 5/200
35/35 [==============================] - 1s 22ms/step - loss: 11.0693 - accuracy: 0.0000e+00 - val_loss: 6.5920 - val_accuracy: 0.0000e+00
Epoch 6/200
35/35 [==============================] - 1s 21ms/step - loss: 18.0048 - accuracy: 0.0000e+00 - val_loss: 11.3270 - val_accuracy: 0.0000e+00
Epoch 7/200
35/35 [==============================] - 1s 22ms/step - loss: 18.5131 - accurac

In [189]:
AlexNet.predict(x_test)

array([[ 29.022205],
       [ 72.90264 ],
       [ 43.010418],
       [ 26.751265],
       [ 38.383804],
       [174.41971 ],
       [ 51.22464 ],
       [ 33.788067],
       [ 42.51954 ],
       [ 79.16757 ],
       [ 25.16471 ],
       [175.73558 ],
       [118.48358 ],
       [ 28.13005 ],
       [ 52.49104 ],
       [ 56.316044],
       [ 67.99605 ],
       [ 27.204859],
       [ 28.120188],
       [ 46.538734],
       [ 26.045826],
       [ 38.69154 ],
       [ 29.131771],
       [119.10367 ],
       [ 36.23856 ],
       [ 36.605072],
       [123.944565],
       [ 38.7561  ],
       [126.956924],
       [ 61.31727 ],
       [111.66885 ],
       [ 39.14001 ],
       [ 65.3413  ],
       [ 46.623207],
       [ 39.369843],
       [119.38499 ],
       [ 26.45031 ],
       [ 66.508354],
       [ 35.1283  ],
       [ 28.221756],
       [ 28.412525],
       [ 26.035883],
       [ 66.70737 ],
       [ 47.659897],
       [113.43178 ],
       [174.54816 ],
       [ 48.01674 ],
       [ 27.1

In [190]:
AlexNet.evaluate(x_test, y_test)

10/10 [==============================] - 0s 5ms/step - loss: 4.1677 - accuracy: 0.0000e+00


[4.167749881744385, 0.0]